In [1]:
import psycopg2
from config import load_config
import pandas as pd
import psycopg2.extras

In [2]:
DATA_DIR = "C:/Projects/Rajk/prog_2/fourth_hw/c4-frontend-barab/data"

In [3]:
DATA_DIR

'C:/Projects/Rajk/prog_2/fourth_hw/c4-frontend-barab/data'

In [4]:
config = load_config()

In [6]:
with psycopg2.connect(**config) as conn:
    print('Connected to the PostgreSQL server.')
    

Connected to the PostgreSQL server.


## Parental guides

In [7]:
data = pd.read_csv("{}/parental-guides.csv".format(DATA_DIR))

In [8]:
data["x"] = data["x"].str.replace(',','')
data["y"] = data["y"].str.replace(',','')

In [9]:
data = data.astype({"x":int})
data = data.astype({"y":int})
data = data.astype({"cat":str})

In [10]:
data.head()

,t,cat,level,x,y,n
0,tt0944947,Sex & Nudity,Severe,7645,8721,7645
1,tt0944947,Violence & Gore,Severe,4106,4471,4106
2,tt0944947,Profanity,Severe,3459,4128,3459
3,tt0944947,"Alcohol, Drugs & Smoking",Moderate,1769,4708,1769
4,tt0944947,Frightening & Intense Scenes,Severe,3163,3889,3163


In [11]:
sql = '''CREATE TABLE parentalguides ( 
t char(20) ,cat char(30),level char(20),x int, y int, n int);'''
delete_sql = '''DROP TABLE parentalguides'''

In [12]:
conn = psycopg2.connect(**config)
cursor = conn.cursor()
cursor.execute(delete_sql)
cursor.execute(sql)
conn.commit()
conn.close()

In [13]:
data.dtypes

t        object
cat      object
level    object
x         int32
y         int32
n         int64
dtype: object

In [14]:
conn = psycopg2.connect(**config)
df_columns = list(data)
# create (col1,col2,...)
columns = ",".join(df_columns)

# create VALUES('%s', '%s",...) one '%s' per column
values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

#create INSERT INTO table (columns) VALUES('%s',...)
insert_stmt = "INSERT INTO {} ({}) {}".format("parentalguides",columns,values)

cur = conn.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, data.values)
conn.commit()
cur.close()

## Series data

In [21]:
data = pd.read_csv("{}/series.csv".format(DATA_DIR))

In [25]:
sql = '''CREATE TABLE series ( 
t char(20) ,title char(50),rating float(5),vote_num int);'''
delete_sql = '''DROP TABLE series'''

In [26]:
conn = psycopg2.connect(**config)
cursor = conn.cursor()
cursor.execute(delete_sql)
cursor.execute(sql)
conn.commit()
conn.close()

In [27]:
conn = psycopg2.connect(**config)
df_columns = list(data)
# create (col1,col2,...)
columns = ",".join(df_columns)

# create VALUES('%s', '%s",...) one '%s' per column
values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

#create INSERT INTO table (columns) VALUES('%s',...)
insert_stmt = "INSERT INTO {} ({}) {}".format("series",columns,values)

cur = conn.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, data.values)
conn.commit()
cur.close()